In [1]:
import toolviper
import xradio
import pathlib
import numba

import numpy as np
import pandas as pd
import calviper as cv

from xradio import measurement_set as ms

In [2]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:200% !important; }</style>"))

In [3]:
if not pathlib.Path.cwd().joinpath("data/gaincaltest2.ps.zarr").exists():
    toolviper.utils.data.download("gaincal.test.zarr", "data")

In [4]:
df_gains = pd.read_csv("gains.csv")
gains = df_gains.gain.apply(complex).to_numpy()
gains.shape

(10,)

In [5]:
df_vis = pd.read_csv("vis.csv")
df_vis.drop(columns=["Unnamed: 0"], inplace=True)

vis = df_vis.vis.apply(complex).to_numpy()

ant_a = df_vis.ant_a.to_numpy()
ant_b = df_vis.ant_b.to_numpy()

In [6]:
from calviper.math.loss import mean_squared_error as mse

def build_vis(vis, index_a, index_b):
    count = 1
    
    X = np.zeros((10, 10), dtype=complex)

    for i in range(45):
        m = index_a[i]
        n = index_b[i]

        X[m, n] = X[m, n]*(count - 1) + vis[i]
        X[m, n] = X[m, n]/count

        X[n, m] = X[m, n].conj()

    return X

In [7]:
X = build_vis(vis, ant_a, ant_b)
X.shape

(10, 10)

In [8]:
solver = cv.math.solver.least_squares.LeastSquaresSolver()

In [9]:
g = solver.solve(
    vis=X,
    iterations=10,
    optimizer=cv.math.optimizer.MeanSquaredError(alpha=0.2),
    stopping=1e-2
)

ValueError: not enough values to unpack (expected 4, got 2)

In [ ]:
ps = ms.open_processing_set("data/gaincaltest2.ps.zarr")

sub_ps = ps.sel(intents="CALIBRATE_DELAY#ON_SOURCE", scan_number=2)
sub_ps.summary()

In [ ]:
sps = sub_ps["gaincaltest2_0"]

In [ ]:
V = sps.VISIBILITY.mean(dim="time").data.compute()

In [ ]:
def build_matrix(vis, index_a, index_b):
    length = index_a.shape[0]
    
    X = np.zeros((length, length), dtype=complex)
    
    for m in index_a:
        for n in index_b[1:]:
            X[m, n] = vis[m]
            X [n, m] = vis[m].conj()

    return X
            

def counter(n):
    counter = 0
    for i in range(n):
        for j in range(i, n):
            if i == j:
                continue
                
            counter += 1

    return counter

In [ ]:
#@numba.njit()
def build_visibility_matrix(array: np.ndarray, index_a: np.ndarray, index_b: np.ndarray) -> np.ndarray:
    """
    Build a visibility matrix from a visibility array with zeros for autocorrelation.
    :param index_b:
    :param index_a:
    :param array: (numpy.ndarray) visibility array
    :return: (np.ndarray) Visibility matrix of dimension N x N.
    """
    # Get the full array length
    size = index_a.shape[0]

    # Calculate the N X N matrix size needed
    dimension = np.unique(index_a).shape[0] + 1

    # Build matrix
    matrix_ = np.zeros((dimension, dimension), dtype=np.complex64)

    for m in range(size):
        i = index_a[m]
        j = index_b[m]

        if i == j:
            continue
        print(f"({i}, {j}): {array[m]}")
        #matrix_[i, j] = array[m]
        #matrix_[j, i] = np.conj(array[m])

    return matrix_

In [ ]:
vis_array = V[:, 0, 0]

index_a, _ = cv.math.tools.encode(sps.baseline_antenna1_name.to_numpy())
index_b, _ = cv.math.tools.encode(sps.baseline_antenna2_name.to_numpy())

V = cv.math.tools.build_visibility_matrix(array=vis_array, index_a=index_a, index_b=index_b)
V.shape

In [ ]:
g = solver.solve(
    vis=V,
    iterations=20,
    optimizer=cv.math.optimizer.MeanSquaredError(alpha=0.2),
    stopping=1e-3
)

In [ ]:
import matplotlib.pyplot as plt

t = np.linspace(1, len(solver.losses), len(solver.losses))

plt.scatter(solver.losses, t)

In [ ]:
np.abs(solver.parameter)